# Albero decisionale per la classificazione del traffico di rete. 
In questo laboratorio, addestriamo un albero decisionale (DT) con traffico di rete benigno e quattro classi di attacchi DDoS dal dataset CIC-DDoS2019 dell’Università del New Brunswick. Il traffico di rete è stato precedentemente pre-elaborato in modo che i pacchetti siano raggruppati in flussi di traffico bidirezionali utilizzando la 5-tupla (IP sorgente, IP destinazione, porta sorgente, porta destinazione, protocollo). Ogni flusso è rappresentato da 21 features (attributi) dell’header dei pacchetti calcolate da un massimo di 1000 pacchetti:

| Feature nr.         | Feature Name |
|---------------------|---------------------|
| 00 | timestamp (mean IAT) | 
| 01 | packet_length (mean)| 
| 02 | IP_flags_df (sum) |
| 03 | IP_flags_mf (sum) |
| 04 | IP_flags_rb (sum) | 
| 05 | IP_frag_off (sum) |
| 06 | protocols (mean) |
| 07 | TCP_length (mean) |
| 08 | TCP_flags_ack (sum) |
| 09 | TCP_flags_cwr (sum) |
| 10 | TCP_flags_ece (sum) |
| 11 | TCP_flags_fin (sum) |
| 12 | TCP_flags_push (sum) |
| 13 | TCP_flags_res (sum) |
| 14 | TCP_flags_reset (sum) |
| 15 | TCP_flags_syn (sum) |
| 16 | TCP_flags_urg (sum) |
| 17 | TCP_window_size (mean) |
| 18 | UDP_length (mean) |
| 19 | ICMP_type (mean) |
| 20 | Packets (counter)|

# Importazione delle librerie necessarie

In [ ]:
# Author: Roberto Doriguzzi-Corin
# Project: Corso di Algoritmi di Machine Learning per la rilevazione di attacchi informatici
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import os
from IPython.display import display, Image
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import metrics 
from util_functions import *

OUTPUT_FILE = "./ddos_tree"
DATASET_FOLDER = "../Dataset/"

# Caricamento del dataset e altre funzioni

In [ ]:
X_train, y_train = load_dataset(DATASET_FOLDER + "/*" + '-train.hdf5')
X_test, y_test = load_dataset(DATASET_FOLDER + "/*" + '-test.hdf5')

feature_names = get_feature_names()
target_names = ['benign', 'dns',  'syn', 'udplag', 'webddos'] 

export_graphviz(
    tree_clf,
    out_file=OUTPUT_FILE + ".dot",
    feature_names=feature_names,
    class_names=target_names,
    rounded=True,
    filled=True
)

# Addestramento dell'albero decisionale
L'albero decisionale puo' essere configurato con diversi parametri. Questi parametri servono per indicare quanti dettagli del training set l'albero deve imparare. I parametri piu' comuni sono:
- ```max_depth```: profondita' dell'albero (numero di strati)
- ```min_samples_split```: Numero minimo di campioni richiesti per dividere un nodo interno in un albero decisionale.
    - **Esempio**: se ```min_samples_split``` è impostato a 50, allora ogni divisione nell’albero deve avere almeno 50 campioni. Ciò significa che solo i nodi con piu' di 50 campioni possono essere divisi in due nodi figli, e i nodi foglia devono avere meno di 50 campioni.
- ```min_samples_leaf```: Numero minimo di campioni richiesti per essere in un nodo foglia.
    - **Esempio**: se ```min_samples_leaf``` è impostato a 20, allora ogni nodo foglia nell’albero deve avere almeno 20 campioni. Ciò significa che, anche se un nodo ha più di 50 campioni, non verrà diviso in due nodi figli se ciò comporterebbe la creazione di un nodo foglia con meno di 20 campioni.

**Suggerimento**: provate a cambiare i parametri della funzione ```DecisionTreeClassifier``` ed i loro valori.

In [ ]:
tree_clf = DecisionTreeClassifier(max_depth=3) # altri parametri sono min_samples_split e min_samples_leaf
tree_clf.fit(X_train,y_train)

# L'albero decisionale addestrato

In [ ]:
os.system("dot -Tpng " + OUTPUT_FILE + ".dot -o " + OUTPUT_FILE + ".png")
display(Image(filename=OUTPUT_FILE + ".png"))

## Classificazione
Ora utilizziamo l’albero decisionale addestrato per classificare campioni di traffico non usati per l'addestramento (il set di test).

In [ ]:
y_pred = tree_clf.predict(X_test)

for y_index in range(y_pred.shape[0]):
    dt_result_string = "" if y_pred[y_index] == y_test[y_index] else " <-- Errore!!! Classificato " + target_names[y_pred[y_index]] + " invece di " + target_names[y_test[y_index]]
    print ("Campione " + str(y_index) + " - " + "Classificato come: " + str(y_pred[y_index]) + " Classe vera (label): " + str(y_test[y_index]) + dt_result_string)


# Analizziamo gli errori del modello
Gli alberi decisionali sono intuitivi e le loro decisioni sono facili da interpretare. Al contrario, come vedremo, le Random Forests o le reti neurali sono generalmente molto piu' difficili da analizzare ed e' piu' compicato capire il motivo delle loro decisioni.

Vediamo dove l’albero decisionale ha sbagliato!

In [ ]:
float_formatter = "{:.2f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})
                    
for y_index in range(y_pred.shape[0]):
    if y_pred[y_index] != y_test[y_index]:
        print("Campione sbagliato n. " + str(y_index) + " (Classificato " + target_names[y_pred[y_index]] + " invece di " + target_names[y_test[y_index]] + ")")
        for feature_index in range(len(feature_names)):
            print (feature_names[feature_index] + ": " + str(X_test[y_index][feature_index]))
        print("")

# Accuratezza
Ora supponiamo di voler capire solo qual'e' il traffico benigno da quello malevolo. Questo ci permette di bloccare tutto cio' che non e' benigno. 

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score, confusion_matrix
np.set_printoptions(precision=2)

y_test_binary = (y_test > 0).astype(int)
y_pred_binary = (y_pred > 0).astype(int)

cm = confusion_matrix(y_test_binary, y_pred_binary)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=['Benigno', 'DDoS'])
disp.plot(cmap='Blues')
plt.title('Confusion Matrix')
plt.show()

# Calcoliamo l'accuratezza del modello
Usate i valori della confusion matrix ottenuta sopra per calcolare l'accuratezza.
Come primo passo, dovete assegnare i valori giusti alle seguenti variabili:
- ```tn``` (true negative)
- ```fp``` (false positive)
- ```fn``` (false negative)
- ```tp``` (true negative)
Nel secondo passo, usate queste variabili per calcolare l'accuratezza.

In [ ]:
tn =
fp =
fn = 
tp =

accuratezza = 
print ("Accuratezza dell'albero decisionale: ", accuratezza)

# Matrice di confusione multi-classe
Ora ritorniamo alle nostre 5 classi di traffic (benigno e 4 attacchi) e analizziamo in dettaglio il risultato della classificazione dell'albero decisionale.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score, classification_report
np.set_printoptions(precision=2)

cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=target_names)
disp.plot(cmap='Blues')
plt.title('Confusion Matrix')
plt.show()